In [1]:
import cv2
import numpy as np
import os
import pandas
import sys
import keras
import pytesseract
from matplotlib import pyplot as plt
from shutil import copyfile
from PIL import Image, ImageChops

Using TensorFlow backend.


In [2]:
def convert_images(filenames,directory,height,width,outdir):
    # Convert image to the same siz

    for file in filenames:
        image = cv2.imread(directory + "/" + file,0)
        # Resize the image.
        resized_image = cv2.resize(image, (width,height))
        # Write it to file
        cv2.imwrite(outdir + "/" + file, resized_image)

    return images

In [3]:
def get_sizes(filenames,directory):
    # Read Images into a Numpy array
    
    # Get the array size
    array_size = len(filenames)
    height = np.array([])
    width = np.array([])
    
    i=0
    for file in filenames:
        image = cv2.imread(directory + "/" + file,0)
        # Get Size
        # Add it to numpy array
        height = np.append(height,image.shape[0])
        width = np.append(width,image.shape[1])
    
    return height,width

In [4]:
def find_rectangle(gray):
 
    ret,thresh = cv2.threshold(gray,100,255,1)
    _,contours,h = cv2.findContours(thresh,1,2)

    biggest_box = np.array([])

    for cnt in contours:
        approx = cv2.approxPolyDP(cnt,0.01*cv2.arcLength(cnt,True),True)
    
        if len(approx)== 4:
            area = cv2.contourArea(cnt)
        
            if area > 1000 and area < 200000:
                biggest_box=cnt
        else:
            continue
            
    return biggest_box

In [5]:
def crop_bottom(image,box):

    x = np.array([])
    y = np.array([])
    for i in box:
        x = np.append(x,i[:,0])
        y = np.append(y,i[:,1])

    x1 = int(np.amin(x))
    y1 = int(np.amin(y))
    x2 = int(np.amin(x))
    y2 = int(np.amax(x))

    cropped = image[x1:y1, x2:y2]
    return cropped

In [6]:
def crop_text_bottomP(image,lines,width,height):
    #print("Probalistic Hough")
    cropped = np.array([])
    for x1,y1,x2,y2 in lines[0]:
        if y1 < int(height*.6) or y2 < int(height*.6):
            continue
        else:
            #print(x1,y1,x2,y2)
            cropped = image[0:y1,0:width]
            cropped = cropped[0:y2,0:width]
    return cropped

In [7]:
def crop_text_bottom(image,lines,width,height):
    #print("Regular Hough")
    cropped = np.array([])
    for rho, theta in lines[0]:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))
        if y1 < int(height*.6) or y2 < int(height*.6):
            continue
        else:
            #cv2.line(image, (x1, y1), (x2, y2), (0, 0, 0), 20)
            #print(x1,y1,x2,y2)
            cropped = image[0:y1,0:width]
            cropped = cropped[0:y2,0:width]

    return cropped

In [8]:
def process_text_image(filename):
    
    image = cv2.imread(inputdir + "/" + filename, 0) # Working - Should Not Change

    height=image.shape[0]
    width=image.shape[1]

    # construct a sharpening filter
    kernel = np.array(([0, -1, 0],[-1, 5, -1],[0, -1, 0]), dtype="int")
    im1 = cv2.filter2D(image, -1, kernel)
    im2 = cv2.medianBlur(image, 3)

    edges1 = cv2.Canny(image,120,200,3)
    edges2 = cv2.Canny(im1,120,200,3)
    edges3 = cv2.Canny(im2,120,200,3)


    # Text needs to be cropped out.  0 Does not contain text, 1 has text
    has_text = 0
    text1 = pytesseract.image_to_string(edges1)
    text2 = pytesseract.image_to_string(edges2)


    if len(text1) > 5:
        has_text=1
    if len(text2) > 5:
        has_text=1
    
    cropped = np.array([])
        
    if has_text == 1:
        # Try Regular Houghlines
        maxLineGap = 1
        maxLineLength = width
        minLineLength = int(width*.6)
        threshold = 260
    
        #Try Regular Hough Lines
        if cropped.size == 0:
            lines1 = cv2.HoughLines(edges1, 1, np.pi/180, threshold)
            if lines1 is not None:
                cropped = crop_text_bottom(image,lines1,width,height)

        if cropped.size == 0:
            lines2 = cv2.HoughLines(edges2, 1, np.pi/180, threshold)
            if lines2 is not None:
                cropped = crop_text_bottom(image,lines2,width,height)
        
        if cropped.size == 0:
            lines3 = cv2.HoughLines(edges3, 1, np.pi/180, threshold)
            if lines3 is not None:
                cropped = crop_text_bottom(image,lines3,width,height)
    
        #Try Probalistic Hough Lines
        if cropped.size == 0:
            linesp1 = cv2.HoughLinesP(edges1,1,np.pi/180,100,minLineLength,maxLineGap)
            if linesp1 is not None:
                cropped = crop_text_bottomP(image,linesp1,width,height)
    
        if cropped.size == 0:
            linesp2 = cv2.HoughLinesP(edges2,1,np.pi/180,100,minLineLength,maxLineGap)
            if linesp2 is not None:
                cropped = crop_text_bottomP(image,linesp2,width,height)

        if cropped.size == 0:
            linesp3 = cv2.HoughLinesP(edges3,1,np.pi/180,100,minLineLength,maxLineGap)
            if linesp3 is not None:
                cropped = crop_text_bottomP(image,linesp3,width,height)
            
            
        if cropped.size != 0:
            return cropped,has_text
        
    return cropped,has_text

In [9]:
def crop_bottom_top(image):
    height=image.shape[0]
    width=image.shape[1]

    #print("Original Dimensions:", height,width)

    maxLineGap = 1
    maxLineLength = width
    minLineLength = int(height*.30)
    threshold=1000

    # Display Image
    #plt.imshow(image, cmap = 'gray', interpolation = 'bicubic')
    #plt.show()

    edges1 = cv2.Canny(image,120,200,3)

    img_th = cv2.adaptiveThreshold(edges1,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,3,2)

    # Display Image
    #plt.imshow(img_th, cmap = 'gray', interpolation = 'bicubic')
    #plt.show()

    linesp1 = cv2.HoughLinesP(img_th,1,np.pi/180,threshold,minLineLength,maxLineGap)
    
    cropped = np.array([])

    if linesp1 is None:
        #print("No Lines found")
        return cropped
    else:
        max_height = int(height*.98)
        min_height = int(height*.02)

        previousmaxy=height
        previousminy=0

        # Crop Bottom Lines
        for line in linesp1:
            for x1,y1,x2,y2 in line:

                if y1 > max_height or y2 > max_height:
                    # We don't want diagonal or vertical lines
                    if y1 != y2:
                        continue
                    else:
                        if y1 < previousmaxy:
                            previousmaxy=y1
                            if cropped.size == 0:
                                cropped = image[0:y1, 0:width]
                            else:
                                height=cropped.shape[0]
                                width=cropped.shape[1]
                                #print(height,width)
                                cropped = cropped[0:y1, 0:width]

        # Crop top Lines
        for line in linesp1:
            for x1,y1,x2,y2 in line:
                if y1 < min_height or y2 < min_height:
                    # We don't want diagonal or vertical lines
                    if y1 != y2:
                        continue
                    else:
                        if y1 > previousminy:
                            previousminy=y1
                            if cropped.size == 0:
                                cropped = image[y1:height, 0:width]
                            else:
                                height=cropped.shape[0]
                                width=cropped.shape[1]
                                cropped = cropped[y1:height,0:width]
        return cropped

In [10]:
def crop_left_right(image):

    height=image.shape[0]
    width=image.shape[1]

    #print("Original Dimensions:", height,width)

    maxLineGap = 1
    maxLineLength = height
    minLineLength = int(height*.30)
    threshold=10
    
    edges1 = cv2.Canny(image,120,200,3)
    #plt.imshow(edges1, cmap = 'gray', interpolation = 'bicubic')
    #plt.show()

    linesp1 = cv2.HoughLinesP(edges1,1,np.pi/2,threshold=threshold,minLineLength=minLineLength,maxLineGap=maxLineGap)
    
    cropped = np.array([])

    if linesp1 is None:
        return cropped
    else:
        max_width = int(width*.95)
        min_width = int(width*.05)

        previousmaxx=width
        previousminx=0

        # Crop Right Frames
        for line in linesp1:
            for x1,y1,x2,y2 in line:
                if x1 > max_width or x2 > max_width:
                    # We don't want diagonal or horizontal lines
                    if x1 != x2:
                        continue
                    else:
                        if x1 < previousmaxx:
                            #print(previousmaxx)
                            #print("Cropping Right: ",x1,y1,x2,y2)
                            previousmaxx=x1
                                
        if previousmaxx != width:
            #print(previousmaxx)
            cropped = image[0:height, 0:previousmaxx]
            
        # Crop Left Frames
        for line in linesp1:
            for x1,y1,x2,y2 in line:
                if x1 < min_width or x2 < min_width:
                    # We don't want diagonal or horizontal lines
                    if x1 != x2:
                        continue
                    else:
                        if x1 > previousminx:
                            #print(previousminx)
                            #print("Cropping Left:",x1,y1,x2,y2)
                            previousminx=x1
                            
        if previousminx != 0: 
            #print(previousminx)
            if cropped.size == 0:
                cropped = image[0:height,previousminx:width]
            else:
                cropped = cropped[0:height,previousminx:width]
                                
    return cropped

In [11]:
def crop_tail(image):
    # Find the tail and crop everything else out
    height=image.shape[0]
    width=image.shape[1]

    dst =  cv2.fastNlMeansDenoising(image,None,10,7,21)
    edges1 = cv2.Canny(dst,100,250,3)

    _,contours, hierarchy = cv2.findContours(edges1, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
    cropped = np.array([])
    if not contours:
        print("No Contours")
    else:
        boxes = []
        for c in contours:
            if c.shape[0] < 100:
                continue
            else:
                (x, y, w, h) = cv2.boundingRect(c)
                boxes.append([x,y, x+w,y+h])

        boxes = np.asarray(boxes)
        if boxes.size != 0:
            left = np.min(boxes[:,0])
            top = np.min(boxes[:,1])
            right = np.max(boxes[:,2])
            bottom = np.max(boxes[:,3])
            cropped = image[top:bottom,left:right]
            
            new_width = cropped.shape[0]
            new_height = cropped.shape[1]
            area = new_width * new_height
            # Make sure they haven't cropped too much
            if area > 90000:
                return cropped
            else:
                return np.array([])
            
    return cropped

In [12]:
inputdir="/home/james/Kaggle/Whale/Data/test/1"
outputdir="/home/james/Kaggle/Whale/Data/test/2"
cropdir="/home/james/Kaggle/Whale/Data/test/3"
taildir="/home/james/Kaggle/Whale/Data/test/4"

In [13]:
filenames = os.listdir(inputdir)
filenames.sort()





In [14]:
full_exif = {}
file_csv = "/home/james/Kaggle/Whale/Data/exif_test.csv"
exif_count = 0
for file in filenames:
    img = Image.open("/home/james/Kaggle/Whale/Data/test/1/" + file)
    width, height = img.size
    try:
        exif = {
            PIL.ExifTags.TAGS[k]: v
            for k, v in img._getexif().items()
            if k in PIL.ExifTags.TAGS
        }
    except:
        full_exif[file] = {'width':width,'height':height}
        continue
        
    if len(exif) != 0:
        full_exif[file] = exif
        full_exif[file]['width'] = width
        full_exif[file]['height'] = height
        
        exif_count += 1
        #print("Exif: ",file)
    else:
        full_exif[file] = {'width':width,'height':height}
        #print("No Exif: ", file)
        continue

print(len(full_exif))
print("Exif Count: ",exif_count)    

#Create list with all exif key names
key_list = []
for exif in full_exif:
    sub_dict = full_exif[exif]
    if len(sub_dict) != 0:
        for key in sub_dict:
            key_list.append(key)

# Add filename to list
key_list.insert(0, 'filename')

# Now instantiate a dataframe with those columns
df = pandas.DataFrame(columns=set(key_list))
    


# Now lets fill this dataframe with exif data
for exif in full_exif:
    sub_dict = full_exif[exif]
    
    if len(sub_dict) != 0:
        sub_dict['filename'] = exif
        df = df.append(sub_dict, ignore_index=True)
    else:
        sub_dict['filename'] = exif
        df = df.append(sub_dict, ignore_index=True)
    
#Set Filename as index
df = df.set_index('filename')
  
df.to_csv(file_csv, sep='\t')
# Write it to CSV file for training later

7960
Exif Count:  0


In [15]:

# Remove Large Text Boxes from bottom of picture.
# This is not perfect. But it takes alot of effort out of an otherwise manual process.
#for file in filenames:
#    new_image,has_text = process_text_image(file)
#    if new_image.size != 0:
#        cv2.imwrite(outputdir + "/" + file,new_image)
#        print(file,": Cropping Image")
#    elif has_text==1:
#        # We still want to identify those with text that didn't get cropped
#        print(file,": Text, no crop")
#        copyfile(inputdir + "/" + file,outputdir + "/" + file )

In [16]:
# Get the average height and width of images so we can resize later
#height,width = get_sizes(filenames,inputdir)
#image_height = int(round(np.average(height)))
#image_width = int(round(np.average(width)))
# Manipulate the images and return the array
#chunk_images = convert_images(filenames,inputdir,image_height,image_width,outputdir)

In [17]:
# Remove Black lines from top and bottom, left and right
#for file in filenames:
#    image = cv2.imread(outputdir + "/" + file,0)
#    cropped = crop_bottom_top(image)
#    if cropped.size == 0:
#        cropped = crop_left_right(image)
#    else:
#        cropped = crop_left_right(cropped)
#        
#    if cropped.size != 0:
#        cv2.imwrite(cropdir + "/" + file,cropped)
#        print(file,": Cropping Image")

11ba5cf47.jpg : Cropping Image
16c3fdc3c.jpg : Cropping Image
25fbd433a.jpg : Cropping Image
352264d65.jpg : Cropping Image
5a5dfde16.jpg : Cropping Image
5bbdedc36.jpg : Cropping Image
5e4ad7f18.jpg : Cropping Image
6acbd8732.jpg : Cropping Image
6b57225c1.jpg : Cropping Image
6b59853ec.jpg : Cropping Image
86a174149.jpg : Cropping Image
876dbab7b.jpg : Cropping Image
8b60216ee.jpg : Cropping Image
97bdef817.jpg : Cropping Image
9ecf3864e.jpg : Cropping Image
a6936d1a8.jpg : Cropping Image
b156bdd13.jpg : Cropping Image
bfb04b85b.jpg : Cropping Image
c031b3ca0.jpg : Cropping Image
c3d47f701.jpg : Cropping Image
d64ae9400.jpg : Cropping Image
d905f4b2a.jpg : Cropping Image
dcb879985.jpg : Cropping Image
e0a35c9fe.jpg : Cropping Image
e1ad5f0de.jpg : Cropping Image
eb34af4ac.jpg : Cropping Image
f57b67033.jpg : Cropping Image
f5d754208.jpg : Cropping Image


In [18]:
# Find the Whales tail
for file in filenames:
    image = cv2.imread(cropdir + "/" + file,0)
    cropped = crop_tail(image)

    if cropped.size != 0:
        cv2.imwrite(taildir + "/" + file,cropped)
        print(file,": Cropping Image")


00028a005.jpg : Cropping Image
000dcf7d8.jpg : Cropping Image
000e7c7df.jpg : Cropping Image
0019c34f4.jpg : Cropping Image
00247bc36.jpg : Cropping Image
0027089a4.jpg : Cropping Image
002de4d94.jpg : Cropping Image
002f52f0c.jpg : Cropping Image
00313e2d2.jpg : Cropping Image
0041a9867.jpg : Cropping Image
004344e9f.jpg : Cropping Image
0048970f9.jpg : Cropping Image
004fa8ff7.jpg : Cropping Image
00512687e.jpg : Cropping Image
006183fb4.jpg : Cropping Image
0061febfc.jpg : Cropping Image
0065d4964.jpg : Cropping Image
00744bd58.jpg : Cropping Image
007886aef.jpg : Cropping Image
008a4bc86.jpg : Cropping Image
009dd6c96.jpg : Cropping Image
00a427ac6.jpg : Cropping Image
00ac0fca6.jpg : Cropping Image
00bb36bcf.jpg : Cropping Image
00ca82e74.jpg : Cropping Image
00ccb53ec.jpg : Cropping Image
00d101f4b.jpg : Cropping Image
00dc02f06.jpg : Cropping Image
00e94fd56.jpg : Cropping Image
00edab267.jpg : Cropping Image
00f44f4e1.jpg : Cropping Image
00ff45291.jpg : Cropping Image
01034a16

0894d33d2.jpg : Cropping Image
089d89af8.jpg : Cropping Image
089ea7ad4.jpg : Cropping Image
08a9d7464.jpg : Cropping Image
08ad8c606.jpg : Cropping Image
08af2949e.jpg : Cropping Image
08d90d7c4.jpg : Cropping Image
08eaeb761.jpg : Cropping Image
08f84d436.jpg : Cropping Image
08f85605f.jpg : Cropping Image
08f9703c2.jpg : Cropping Image
08fe4393c.jpg : Cropping Image
0901ea8df.jpg : Cropping Image
090697c46.jpg : Cropping Image
09083e03b.jpg : Cropping Image
091995bde.jpg : Cropping Image
091a88443.jpg : Cropping Image
09232fe8c.jpg : Cropping Image
09243b84a.jpg : Cropping Image
092947f89.jpg : Cropping Image
093e965a3.jpg : Cropping Image
0941de387.jpg : Cropping Image
094d2354f.jpg : Cropping Image
0956ac81e.jpg : Cropping Image
095ce0289.jpg : Cropping Image
09629641e.jpg : Cropping Image
09685092f.jpg : Cropping Image
0969f46de.jpg : Cropping Image
096e5b78c.jpg : Cropping Image
097fdfbea.jpg : Cropping Image
097ffdb27.jpg : Cropping Image
0982ee2f3.jpg : Cropping Image
09897a49

114030f79.jpg : Cropping Image
1145fe373.jpg : Cropping Image
114f2d946.jpg : Cropping Image
11664decf.jpg : Cropping Image
116b6690f.jpg : Cropping Image
1171cf967.jpg : Cropping Image
1173d784c.jpg : Cropping Image
1174922af.jpg : Cropping Image
117d8880c.jpg : Cropping Image
11850b2ea.jpg : Cropping Image
1187f0e93.jpg : Cropping Image
118c7fad8.jpg : Cropping Image
119983f6d.jpg : Cropping Image
119a62706.jpg : Cropping Image
119f9d3f1.jpg : Cropping Image
11ab6d4aa.jpg : Cropping Image
11abeafd0.jpg : Cropping Image
11ba5cf47.jpg : Cropping Image
11beba0df.jpg : Cropping Image
11c3a6621.jpg : Cropping Image
11c4e9112.jpg : Cropping Image
11cd53b5c.jpg : Cropping Image
11ce17042.jpg : Cropping Image
11d369040.jpg : Cropping Image
11d4b7555.jpg : Cropping Image
11dedbc16.jpg : Cropping Image
11e0bcf3a.jpg : Cropping Image
11e15e585.jpg : Cropping Image
11ed73eb4.jpg : Cropping Image
11f1f53fb.jpg : Cropping Image
11f33db3a.jpg : Cropping Image
11f6318e3.jpg : Cropping Image
11fb91e7

19c5a63ea.jpg : Cropping Image
19c779ca5.jpg : Cropping Image
19cc7557f.jpg : Cropping Image
19e06a6bd.jpg : Cropping Image
19e56cba6.jpg : Cropping Image
19e987165.jpg : Cropping Image
1a099d8b8.jpg : Cropping Image
1a175f0fb.jpg : Cropping Image
1a288300a.jpg : Cropping Image
1a4b25e55.jpg : Cropping Image
1a51902e8.jpg : Cropping Image
1a5550c9a.jpg : Cropping Image
1a577b69c.jpg : Cropping Image
1a66a8b1b.jpg : Cropping Image
1a72251b0.jpg : Cropping Image
1a759c77d.jpg : Cropping Image
1a7b8b905.jpg : Cropping Image
1a8302bee.jpg : Cropping Image
1aa7b651a.jpg : Cropping Image
1aadef38f.jpg : Cropping Image
1ab291651.jpg : Cropping Image
1ab7ab90a.jpg : Cropping Image
1ac218498.jpg : Cropping Image
1ac811a9e.jpg : Cropping Image
1ace6f304.jpg : Cropping Image
1ad8b4ad7.jpg : Cropping Image
1adc83ebc.jpg : Cropping Image
1ae50f9e1.jpg : Cropping Image
1ae8d2689.jpg : Cropping Image
No Contours
1af7d183c.jpg : Cropping Image
1afea7c23.jpg : Cropping Image
1b0954425.jpg : Cropping Im

23892fdc3.jpg : Cropping Image
238f7034e.jpg : Cropping Image
239088895.jpg : Cropping Image
23930c511.jpg : Cropping Image
23974ab77.jpg : Cropping Image
23a07e939.jpg : Cropping Image
23c60e8e8.jpg : Cropping Image
23d88c0ea.jpg : Cropping Image
23e3c8d8e.jpg : Cropping Image
23ec8dbf0.jpg : Cropping Image
23fa57e07.jpg : Cropping Image
23fae540b.jpg : Cropping Image
23fe8fa38.jpg : Cropping Image
23fee6d36.jpg : Cropping Image
2402428f3.jpg : Cropping Image
24121b962.jpg : Cropping Image
24157b8ce.jpg : Cropping Image
2426118d7.jpg : Cropping Image
2426d7b04.jpg : Cropping Image
242d3c556.jpg : Cropping Image
243746112.jpg : Cropping Image
2442a5353.jpg : Cropping Image
24475ba59.jpg : Cropping Image
244c50b07.jpg : Cropping Image
24543849b.jpg : Cropping Image
245f0fdb0.jpg : Cropping Image
24688b414.jpg : Cropping Image
246a3b660.jpg : Cropping Image
24749096f.jpg : Cropping Image
247b453da.jpg : Cropping Image
2488c92cd.jpg : Cropping Image
24ab3e86a.jpg : Cropping Image
24b6f116

...

d2330a6a7.jpg : Cropping Image
d23376dba.jpg : Cropping Image
d23c2263f.jpg : Cropping Image
d24ba1178.jpg : Cropping Image
d25689c34.jpg : Cropping Image
d25989728.jpg : Cropping Image
d25bc885a.jpg : Cropping Image
d25be2116.jpg : Cropping Image
d2712bce3.jpg : Cropping Image
d27486c76.jpg : Cropping Image
d2751b760.jpg : Cropping Image
d27634ad5.jpg : Cropping Image
d29acc5f4.jpg : Cropping Image
d29d81d1a.jpg : Cropping Image
d2a5d103b.jpg : Cropping Image
d2b3f795d.jpg : Cropping Image
d2bada615.jpg : Cropping Image
d2bca8b4b.jpg : Cropping Image
d2c2adbc9.jpg : Cropping Image
d2d4d7b03.jpg : Cropping Image
d2d8ce174.jpg : Cropping Image
d2e08d5fe.jpg : Cropping Image
d2e39105a.jpg : Cropping Image
d2eeb9436.jpg : Cropping Image
d2f17d90f.jpg : Cropping Image
d2f80f85b.jpg : Cropping Image
d300d71f3.jpg : Cropping Image
d30593f94.jpg : Cropping Image
d3130dc91.jpg : Cropping Image
d31310f97.jpg : Cropping Image
d31bd1979.jpg : Cropping Image
d31f10647.jpg : Cropping Image
d3336903

dbf3e997c.jpg : Cropping Image
dbff2f5fc.jpg : Cropping Image
dc063861e.jpg : Cropping Image
dc0838dd9.jpg : Cropping Image
dc0f29f9f.jpg : Cropping Image
dc146aba1.jpg : Cropping Image
dc17e4101.jpg : Cropping Image
dc1d40ae6.jpg : Cropping Image
dc1fb8b8f.jpg : Cropping Image
dc2196c11.jpg : Cropping Image
dc2283485.jpg : Cropping Image
dc32bb7c5.jpg : Cropping Image
dc4a40c54.jpg : Cropping Image
dc4c39727.jpg : Cropping Image
dc59d0d48.jpg : Cropping Image
dc6425875.jpg : Cropping Image
dc6b318d1.jpg : Cropping Image
dc732864d.jpg : Cropping Image
dc8f3fd58.jpg : Cropping Image
dc8f729b0.jpg : Cropping Image
dc9477451.jpg : Cropping Image
dc98417ae.jpg : Cropping Image
dca0491d8.jpg : Cropping Image
dca8b694c.jpg : Cropping Image
dcac5687f.jpg : Cropping Image
dcb2c0fe0.jpg : Cropping Image
dcb501be7.jpg : Cropping Image
dcc0a21c0.jpg : Cropping Image
dcc8bf836.jpg : Cropping Image
dcd91ee44.jpg : Cropping Image
dce1fd759.jpg : Cropping Image
dcf360d8f.jpg : Cropping Image
dcf371b3

e4b07a9a4.jpg : Cropping Image
e4d71742b.jpg : Cropping Image
e4e47e7a8.jpg : Cropping Image
e5006794b.jpg : Cropping Image
e500d5b9c.jpg : Cropping Image
e501d0415.jpg : Cropping Image
e505b8341.jpg : Cropping Image
e5100ad9a.jpg : Cropping Image
e51030771.jpg : Cropping Image
e515a13db.jpg : Cropping Image
e524eade3.jpg : Cropping Image
e52e6188c.jpg : Cropping Image
e5421baf5.jpg : Cropping Image
e547ebb40.jpg : Cropping Image
e54b58074.jpg : Cropping Image
e5513f5db.jpg : Cropping Image
e55cd77a2.jpg : Cropping Image
e56e85cfb.jpg : Cropping Image
e56ff3ec5.jpg : Cropping Image
e57463d0f.jpg : Cropping Image
e57568d51.jpg : Cropping Image
e58d89910.jpg : Cropping Image
e59c4c775.jpg : Cropping Image
e5a324d37.jpg : Cropping Image
e5a4fdc4a.jpg : Cropping Image
e5a746357.jpg : Cropping Image
e5c060961.jpg : Cropping Image
e5d4878fd.jpg : Cropping Image
e5e5320f9.jpg : Cropping Image
e5ece9e60.jpg : Cropping Image
e608cd4c3.jpg : Cropping Image
e60b3e96d.jpg : Cropping Image
e61e5b53

ef4e7f5e9.jpg : Cropping Image
ef4f5eb4a.jpg : Cropping Image
ef535e775.jpg : Cropping Image
ef59ca55e.jpg : Cropping Image
ef60d186c.jpg : Cropping Image
ef741b3d5.jpg : Cropping Image
ef766d04d.jpg : Cropping Image
ef9b75685.jpg : Cropping Image
efac7e263.jpg : Cropping Image
efb5c2c54.jpg : Cropping Image
efc8b6385.jpg : Cropping Image
efc9b26f5.jpg : Cropping Image
efc9c5469.jpg : Cropping Image
efd1ae9d2.jpg : Cropping Image
efd361174.jpg : Cropping Image
efd46634d.jpg : Cropping Image
efddaaf19.jpg : Cropping Image
efed7c412.jpg : Cropping Image
effe7813c.jpg : Cropping Image
f00ad3ef5.jpg : Cropping Image
f02bc8efb.jpg : Cropping Image
f02c38230.jpg : Cropping Image
f02f06823.jpg : Cropping Image
f030c1ffc.jpg : Cropping Image
f0335d0de.jpg : Cropping Image
f05412ee6.jpg : Cropping Image
f058d3e72.jpg : Cropping Image
f0600fb6f.jpg : Cropping Image
f062bdc81.jpg : Cropping Image
f06b9e0a4.jpg : Cropping Image
f074d2fb2.jpg : Cropping Image
f09c1a4dc.jpg : Cropping Image
f0a48542

f880c2c31.jpg : Cropping Image
f882bf52d.jpg : Cropping Image
f88927b97.jpg : Cropping Image
f88abc983.jpg : Cropping Image
f8952bbb6.jpg : Cropping Image
f89d11be0.jpg : Cropping Image
f89e16a95.jpg : Cropping Image
f8a016058.jpg : Cropping Image
f8a0f4e82.jpg : Cropping Image
f8a395ffc.jpg : Cropping Image
f8b18f884.jpg : Cropping Image
f8c1fc67f.jpg : Cropping Image
f8c94dafe.jpg : Cropping Image
f8cbf170f.jpg : Cropping Image
f8e70076e.jpg : Cropping Image
f8f5e8ee4.jpg : Cropping Image
f8fb39d6a.jpg : Cropping Image
f8ff9a65a.jpg : Cropping Image
f906d7be9.jpg : Cropping Image
f90a3894b.jpg : Cropping Image
f90bef87f.jpg : Cropping Image
f90dd5ddd.jpg : Cropping Image
f91a9e58a.jpg : Cropping Image
f91ce31ab.jpg : Cropping Image
f9204d66f.jpg : Cropping Image
f923fd696.jpg : Cropping Image
f9293c32b.jpg : Cropping Image
f92e6568a.jpg : Cropping Image
f9344b448.jpg : Cropping Image
f937b553e.jpg : Cropping Image
f93d02bda.jpg : Cropping Image
f94271a9f.jpg : Cropping Image
f95a8e92

In [19]:
image = cv2.imread(taildir + "/" + "09e89d3d0.jpg",0)
print(image.shape)

AttributeError: 'NoneType' object has no attribute 'shape'

In [21]:
filenames2 = os.listdir(taildir)
filenames2.sort()

not_cropped = []
print(type(not_cropped))
for file1 in filenames:
    not_found = 0
    for file2 in filenames2:
        if file1 == file2:
            not_found=1
    if not_found == 0:
        copyfile(cropdir + "/" + file1, taildir + "/" + "not/" + file1)
        not_cropped.append(file1)

print(len(not_cropped))

<class 'list'>
987


In [59]:
help(cv2.medianBlur)

Help on built-in function medianBlur:

medianBlur(...)
    medianBlur(src, ksize[, dst]) -> dst
    .   @brief Blurs an image using the median filter.
    .   
    .   The function smoothes an image using the median filter with the \f$\texttt{ksize} \times
    .   \texttt{ksize}\f$ aperture. Each channel of a multi-channel image is processed independently.
    .   In-place operation is supported.
    .   
    .   @note The median filter uses #BORDER_REPLICATE internally to cope with border pixels, see #BorderTypes
    .   
    .   @param src input 1-, 3-, or 4-channel image; when ksize is 3 or 5, the image depth should be
    .   CV_8U, CV_16U, or CV_32F, for larger aperture sizes, it can only be CV_8U.
    .   @param dst destination array of the same size and type as src.
    .   @param ksize aperture linear size; it must be odd and greater than 1, for example: 3, 5, 7 ...
    .   @sa  bilateralFilter, blur, boxFilter, GaussianBlur



In [37]:
help(cv2.Canny)

Help on built-in function Canny:

Canny(...)
    Canny(image, threshold1, threshold2[, edges[, apertureSize[, L2gradient]]]) -> edges
    .   @brief Finds edges in an image using the Canny algorithm @cite Canny86 .
    .   
    .   The function finds edges in the input image and marks them in the output map edges using the
    .   Canny algorithm. The smallest value between threshold1 and threshold2 is used for edge linking. The
    .   largest value is used to find initial segments of strong edges. See
    .   <http://en.wikipedia.org/wiki/Canny_edge_detector>
    .   
    .   @param image 8-bit input image.
    .   @param edges output edge map; single channels 8-bit image, which has the same size as image .
    .   @param threshold1 first threshold for the hysteresis procedure.
    .   @param threshold2 second threshold for the hysteresis procedure.
    .   @param apertureSize aperture size for the Sobel operator.
    .   @param L2gradient a flag, indicating whether a more accurate \

In [310]:
help(cv2.HoughLinesP)

Help on built-in function HoughLinesP:

HoughLinesP(...)
    HoughLinesP(image, rho, theta, threshold[, lines[, minLineLength[, maxLineGap]]]) -> lines
    .   @brief Finds line segments in a binary image using the probabilistic Hough transform.
    .   
    .   The function implements the probabilistic Hough transform algorithm for line detection, described
    .   in @cite Matas00
    .   
    .   See the line detection example below:
    .   @include snippets/imgproc_HoughLinesP.cpp
    .   This is a sample picture the function parameters have been tuned for:
    .   
    .   ![image](pics/building.jpg)
    .   
    .   And this is the output of the above program in case of the probabilistic Hough transform:
    .   
    .   ![image](pics/houghp.png)
    .   
    .   @param image 8-bit, single-channel binary source image. The image may be modified by the function.
    .   @param lines Output vector of lines. Each line is represented by a 4-element vector
    .   \f$(x_1, y_1, x_2, y

In [99]:
help(cv2.HoughLines)


Help on built-in function HoughLines:

HoughLines(...)
    HoughLines(image, rho, theta, threshold[, lines[, srn[, stn[, min_theta[, max_theta]]]]]) -> lines
    .   @brief Finds lines in a binary image using the standard Hough transform.
    .   
    .   The function implements the standard or standard multi-scale Hough transform algorithm for line
    .   detection. See <http://homepages.inf.ed.ac.uk/rbf/HIPR2/hough.htm> for a good explanation of Hough
    .   transform.
    .   
    .   @param image 8-bit, single-channel binary source image. The image may be modified by the function.
    .   @param lines Output vector of lines. Each line is represented by a 2 or 3 element vector
    .   \f$(\rho, \theta)\f$ or \f$(\rho, \theta, \votes)\f$ . \f$\rho\f$ is the distance from the coordinate origin \f$(0,0)\f$ (top-left corner of
    .   the image). \f$\theta\f$ is the line rotation angle in radians (
    .   \f$0 \sim \textrm{vertical line}, \pi/2 \sim \textrm{horizontal line}\f$ ).
   